# Úkol č. 1 - vizualizace dat a web scraping (do 20. října)

  * V rámci tohoto úkolu musíte stáhnout dat z webu (tzv. _web scraping_, velmi základní) a následně data zpracovat a vizualizovat.
  * Cílem bude stáhnout data ze serveru https://www.volby.cz týkající se voleb do zastupitelstva Vámi vybraného (většího) města, uložit data o závěrečných pracích v tabulkovém formátu a pak vymyslet vizualizace a zobrazení dat, které umožní orientaci v těchto datech a zvýrazní zajímavé informace a zobrazit přehledně časový vývoj různých veličin.
 
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Výběr zdroje dat

Vyberte si větší město, které má zastupitelstvo druhu 3 (Zastupitelstvo statutárního města) a strojově stáhněte informace o stranách a kandidátkách z následujících let:
 * [2002](https://www.volby.cz/pls/kv2002/kv12?xjazyk=CZ&xid=0), [2006](https://www.volby.cz/pls/kv2006/kv12?xjazyk=CZ&xid=0), [2010](https://www.volby.cz/pls/kv2010/kv12?xjazyk=CZ&xid=0), [2014](https://www.volby.cz/pls/kv2014/kv12?xjazyk=CZ&xid=0) a [2018](https://www.volby.cz/pls/kv2018/kv12?xjazyk=CZ&xid=0).
 

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * Strojově stáhněte data pro vybrané město a uložte je všechny do (asi dvou) přehledných tabulek ve formátu _csv_.
  * Data musí obsahovat _alespoň_ toto:
    * Vývoj výsledků (v procentech i počtu hlasů) pro jednotlivé strany v jednotlivých letech.
    * Seznam všech kandidátů všech stran v jednotlivých letech, u kandidáta by mělo být zaznamenáno: jméno, věk v době voleb, navrhující strana, politická příslušnost, volební zisk (procento i počet hlasů), pořadí na kandidátce, pořadí zvolení, jestli získal mandát (tyto informace získáte souhrnně ve _jmenných seznamech_).
  * V druhé části Vašeho Jupyter notebooku pracujte s těmito tabulkami načtenými z _csv_ souboru (aby opravující nemusel spouštět stahování z webu).
  * Tabulky ve formátu _csv_ také odevzdejte.
  * S využitím vybraných nástrojů zpracujte data a vymyslete vizualizace a grafy, aby bylo vidět následující:
    * Časový vývoj (po rocích voleb) počtu kandidujících stran i lidí a to celkově i po jednotlivých stranách (ve volbách, kterých se daná strana účastnila).
    * Věkovou strukturu kandidátů celkově i za jednotlivé strany a vývoj této struktury během jednotlivých voleb.
    * Časový vývoj volební účasti a volebních výsledků jednotlivých stran.
    * Časový vývoj podílu kandidujících s titulem a bez titulu.

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +2 body) U titulů se pokuste rozlišit i různé stupně vzdělání: bakalářský, magisterský, doktorský a vyšší, vojenská hodnost atp. Zkuste odhadnout i podíl žen na kandidátkách.
  * (až +4 body) Pokuste se u jednotlivých kandidátů zjistit, zda kandidovali ve více volbách. Najděte 10 nejpilnějších kandidátů a vypište jejich volební zisky a za jaké strany kandidovali.
  * (až +2 body) Najděte nějaký balíček, který Vám dovolí do Vašeho notebooku zavést interaktivní prvky, např. si vyberete v select-boxu stranu a Váš notebook zobrazí grafy pouze pro ni atp.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) se staženými daty.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

# Stáhnutí dat - Město Plzeň
## Politické strany
### Načtení dat

In [141]:
import pandas as pd
from IPython.display import display
from pandas import MultiIndex
urls = {
    2002: "https://www.volby.cz/pls/kv2002/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=3203&xobec=554791",
    2006: "https://www.volby.cz/pls/kv2006/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=3203&xobec=554791&xstat=0&xvyber=0",
    2010: "https://www.volby.cz/pls/kv2010/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=3203&xobec=554791&xstat=0&xvyber=0",
    2014: "https://www.volby.cz/pls/kv2014/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=3203&xobec=554791&xstat=0&xvyber=0",
    2018: "https://www.volby.cz/pls/kv2018/kv1111?xjazyk=CZ&xid=0&xdz=3&xnumnuts=3203&xobec=554791&xstat=0&xvyber=0",
}

dfs = {}
for i in urls.keys():
    dfs[i] = pd.read_html(urls[i], flavor='html5lib', decimal=',', thousands=' ')[1]
    if i == 2002:
        dfs[i].drop(columns=[
                "Početkandidátů", "Přepočtenýzákladdle počtukandidátů", "Přepočtené % plat. hlasů", "Početmandátů", 
                "Podíly hlasů", ("Kandidátní listina", "Kandidátní listina"),], inplace=True)
        
    else:
        dfs[i].drop(columns=[
                "Početkandidátů", "Přepočtenýzákladdle počtukandidátů", "Přepočtené %platných hlasů", "Početmandátů", 
                "Podílyhlasů", ("Kandidátní listina", "číslo")], inplace=True)
    dfs[i].columns = ["Název strany", "Počet hlasů abs {year}".format(year=i), "Počet hlasů v % {year}".format(year=i)]


### Spojení dat

In [150]:
data = dfs[2002]
for df in dfs.values():
    data = data.merge(df,how="outer")
data.fillna(0, inplace = True)

### Uložení výsledků do csv

In [145]:
#data.to_csv("partyVotes.csv", encoding="windows-1250")
data.to_csv("partyVotes.csv", encoding="utf-8")

## Jednotlivci
### Načtení dat

In [119]:
titles = {'plk.', 'dipl.', 'ped.', 'um.', 'PhDr.Ing.Mgr.et', 'M.A.', 'MUDr.', 'LL.M.', 'M.Sc.', 'RSDr.', 'JUDr.', 'prom.',
          'ThMgr.', 'prom.ped.', 'Arch.', 'Akad.', 'doc.', 'PhDr.', 'Mgr.', 'Mgr.etMgr', 'Ph.D.', 'MgA.', 'Prof.', 'Th.D.',
          'prof.', 'MSc.', 'MVDr.', 'Doc.', 'Bc.', 'Dr.', 'et', 'arch.', 'ing.', 'CSc.', 'Ing.', 'MPA', 'RNDr.', 'DrSc.',
          'FICS.', 'MBA', 'dr.h.c.', 'DSc.', 'DiS.', 'PaedDr.', 'King'
}
# Titles were obtained by script underneath, applied to all years all other than 2002, where title is separated
# and than manually cleaned from third names etc.
# for s in tmp["Jmeno"]:
#    t = s.split(" ")
#    for st, i in zip(t, range(len(t))):
#        if i > 1:
#            titles.add(st.replace(",", ""))


In [157]:
urls = {
    2002: 'https://www.volby.cz/pls/kv2002/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=3203&xobec=554791&xstrana=0',
    2006: 'https://www.volby.cz/pls/kv2006/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=3203&xobec=554791&xstrana=0',
    2010: 'https://www.volby.cz/pls/kv2010/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=3203&xobec=554791&xstrana=0',
    2014: 'https://www.volby.cz/pls/kv2014/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=3203&xobec=554791&xstrana=0',
    2018: 'https://www.volby.cz/pls/kv2018/kv21111?xjazyk=CZ&xid=0&xv=11&xdz=3&xnumnuts=3203&xobec=554791&xstrana=0',
}

dfs = {}

for i in urls.keys():
    dfs[i] = pd.read_html(urls[i], flavor='html5lib', decimal=',', thousands=' ')[0]
    if i == 2002:
        dfs[i].drop(columns=("Kandidátnílistina"), inplace=True)
        
    else:
        dfs[i].drop(columns=("Kandidátní listina"), inplace=True)
        dfs[i].columns = [
            "Pořadí", "Jmeno", "Věk", "Navrhující strana", "Politická příslušnost", 
            "Hlasy abs.", "Hlasy v %", "Pořadí zvolení", "Mandát"]
        t = []
        n = []
        for names in dfs[i]["Jmeno"]:
            names_separated = names.split(" ")
            titles_to_join = []
            for name in names_separated:
                if name in titles:
                    titles_to_join.append(name)
            t.append(" ".join(titles_to_join))
            n.append(" ".join(names_to_join))
        dfs[i].insert(2, "Titul", t)
        dfs[i]["Jmeno"] = n
    dfs[i].columns = [
        "Pořadí na kandidátce v {year}".format(year=i), "Příjmení, jméno", "Tituly", "Věk", "Navrhující strana",
        "Politická příslušnost","Hlasy abs. {year}".format(year=i), "Hlasy v % {year}".format(year=i),
        "Pořadí zvolení v {year}".format(year=i), "Mandát v {year}".format(year=i)]
    dfs[i]["Mandát v {year}".format(year=i)].fillna("-", inplace=True)
    dfs[i].fillna("", inplace=True)

### Spojení dat

In [158]:
data = dfs[2002]
for df in dfs.values():
    data = data.merge(df,how="outer")
data.fillna(0, inplace=True)
data.head(20)

,Pořadí na kandidátce v 2002,"Příjmení, jméno",Tituly,Věk,Navrhující strana,Politická příslušnost,Hlasy abs. 2002,Hlasy v % 2002,Pořadí zvolení v 2002,Mandát v 2002,...,Pořadí na kandidátce v 2014,Hlasy abs. 2014,Hlasy v % 2014,Pořadí zvolení v 2014,Mandát v 2014,Pořadí na kandidátce v 2018,Hlasy abs. 2018,Hlasy v % 2018,Pořadí zvolení v 2018,Mandát v 2018
0,29.0,Adamec Pavel,MUDr.,35,US-DEU,US-DEU,2 474,2.08,26,-,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0
1,45.0,Aloy Zdeněk,Ing.,70,ČSSD,ČSSD,7 283,1.80,35,-,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0
2,3.0,Anderle Jan,Ing.,46,PVP,PVP,5 358,2.92,3,*,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0
3,43.0,Andrlík Roman,,35,ČSSD,ČSSD,7 665,1.89,33,-,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0
4,16.0,Anton Jiří,,27,SZ,BEZPP,1 343,1.86,,-,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0
5,13.0,Arnošt Patrik,,30,US-DEU,US-DEU,2 476,2.08,10,-,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0
6,17.0,Aschenbrenner Lumír,Ing.,42,ODS,ODS,17 550,2.36,3,*,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0
7,47.0,Aust Jan,,71,KSČM,KSČM,5 068,1.94,39,-,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0
8,34.0,Babnič Jan,,48,ČSSD,ČSSD,8 220,2.03,24,-,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0
9,40.0,Ballýová Hana,,61,KSČM,KSČM,5 405,2.07,32,-,...,0.0,0,0.0,0,0,0.0,0,0.0,0,0


### Uložení  do csv

In [ ]:
data.to_csv("partyVotes.csv", encoding="utf-8")